# Moe's Bar Lab

### Introduction

In this lesson, we'll apply our knowledge of optimizing joins a database that stores information about customer orders at a bar.

### Loading Our Data

Then we can connect to our database with the following.

In [1]:
import sqlite3
conn = sqlite3.connect('./moes_bar.db')
cursor = conn.cursor()

Next we load our data.

In [2]:
import pandas as pd
root_url = "https://raw.githubusercontent.com/jigsawlabs-student/curriculum-images/main/has-many-through-bar/data/"
names = ['bartenders', 'customers', 'drinks', 'orders', 'ingredients', 'ingredients_drinks']
loaded_dfs = [pd.read_csv(f'{root_url}{name}.csv') for name in names]

In [4]:
for index, name in enumerate(names):
    loaded_dfs[index].to_sql(f'{name}', conn, index = False, if_exists = 'replace')

### Exploring our Database

Now we list of all of the tables with the following.

In [6]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('bartenders',),
 ('customers',),
 ('drinks',),
 ('orders',),
 ('ingredients',),
 ('ingredients_drinks',)]

And then we can see the details of a particular table with the following.

In [28]:
cursor.execute("pragma table_info(customers)")
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'hometown', 'TEXT', 0, None, 0),
 (3, 'birthyear', 'INTEGER', 0, None, 0)]

### Optimizing Joins

Optimize the following queries.

1. Number of customers placing orders

In [9]:
import pandas as pd
query = """
select count(distinct(customers.id)) customers_placing_orders
from orders 
join customers
on orders.customer_id = customers.id 
"""
pd.read_sql(query, conn)

,customers_placing_orders
0,3


2. Bartender with the most orders

In [12]:
query = """
select bartenders.id as bartender_id, count(*) num_orders 
from orders 
join bartenders
on orders.bartender_id = bartenders.id 
group by bartenders.id
"""
pd.read_sql(query, conn)

,bartender_id,num_orders
0,1,5
1,2,2
2,3,2


3. Bartender name with the most orders

In [15]:
query = """
select bartenders.name as bartender_name, count(*) num_orders 
from orders 
join bartenders
on orders.bartender_id = bartenders.id 
group by bartenders.id order by num_orders desc limit 1 
"""
pd.read_sql(query, conn)

,bartender_name,num_orders
0,moe,5


4. Customers who ordered the most expensive drinks

In [17]:
query = """
select customers.name, drinks.name, drinks.price from drinks
join orders on drinks.id = orders.drink_id
join customers on customers.id = orders.customer_id 
where drinks.price = (select max(price) from drinks)
"""

pd.read_sql(query, conn)

,name,name,price
0,lisa simpson,ice cream float,8
